# Webscraping des données sur le site Laforet

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from lxml import html
import time

In [100]:

#defining a scraping function 
def scraper(url):

    #initialisation du dataframe 
    df = pd.DataFrame(columns=['lien', 'titre','surface','nb_pieces',
                            'nb_chambres','annee_constr','nb_features_int',
                            'features_int','feature_ext','terrain','prix'])

    #df = pd.DataFrame(columns=['lien', 'titre','surface','nb_chambres',
    #                        'prix'])

    #scraping: on commence par recolter les informations de la page principale
    rs = requests.Session() #creation d'une session persistante
    response = rs.get(url) #recolte d'info de l'url de la page
    soup = BeautifulSoup(response.text, "html.parser") #analyse de l'info dans une 'soupe'

    #on cherche l'information de chaque annonce 
    for annonce in soup.find_all("div", {"class": "apartment-card h-100"}):
        time.sleep(0.5)
        i = annonce.find("a",{"class":"card-bottom"})
        link = ("https://www.laforet.com" + str((i['href'])))  
        

        #recherche des informations par page en partant de 'link'
        page= requests.get(link)
        tree= html.fromstring(page.content)

        #On cherche la ville et le type de logement, contenus dans le titre
        try:
            title=tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[1]/div[1]/div[1]/div/h1/text()')
        except:
            title=''


        # Prix du bien
        try:
            price = tree.xpath("//div[@class='property__price w-100']/text()")
        except:
            price = ''


        #surface
        try:
            surface = tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]/text()')
        except:
            surface=''

        #nombre de pieces
        try:
            nb_pieces = tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[1]/div[2]/div[2]/div[2]/div[2]/text()')
        except:
            nb_pieces=''

        #nombre de chambres
        try:
            chambres = tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[1]/div[2]/div[2]/div[2]/div[3]/text()')
        except:
            chambres =''

        #annee de construction
        try:
            annee_constr = tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[4]/div[4]/div/div/div[1]/div/span[2]/text()')
        except:
            annee_constr = ''


        #les elements interieurs qui apportent ue valeur au bien
        #on commence par compter le nombre de features
        main_class = tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[4]/div[2]/div/div')
        sub_classes = main_class[0].xpath('.//div')
        nb_features_int=(len(sub_classes))

        #On recupere le texte correspondant aux features
        try:
            features_int=main_class[0].text_content()
        except:
            features_int=''

        #les elements exterieurs (typiquement piscine)
        try:
            feature_ext = tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[4]/div[3]/div/div/div/div/span[2]/text()')
            if feature_ext is None or feature_ext == []:
                feature_ext = ''
        except:
            feature_ext=''

        # Ajout d'une variable dummy portant sur la presence d'un terrain ou pas
        try:
            feature_ext2 = tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[4]/div[1]/div/div/div[3]/div/span[2]')

            if len(feature_ext2)>0:
                text_content=feature_ext2[0].text_content()
                if "terrain" in text_content:
                    terrain=1
                else:
                    terrain=0
            else:
                terrain=0
        except:
            terrain=''

        
        #on finit par ajouter les caracteristiques de la ville du bien 
        


        
        new_row = pd.DataFrame({'lien': link, 'titre': title, 'surface': surface, 'nb_pieces': nb_pieces,
                            'nb_chambres': chambres,'annee_constr': annee_constr,'nb_features_int': nb_features_int ,
                            'features_int': features_int,'feature_ext': feature_ext,'terrain': terrain, 'prix': price})

        #new_row = pd.DataFrame({'lien': link, 'titre': title, 'surface': surface, 
         #                   'nb_chambres': chambres,'prix':price})
    
    # Append the new row to the DataFrame
        df = pd.concat([df, new_row], ignore_index=True)
    return df
    


In [9]:
#defining a scraping function 
def scraper2(url):

    #initialisation du dataframe
    cols = ['lien', 'titre','surface','nb_pieces',
                            'nb_chambres','annee_constr','nb_features_int',
                            'features_int','features_ext','terrain','prix']
    df = pd.DataFrame(columns=cols)

    #scraping: on commence par recolter les informations de la page principale
    rs = requests.Session() #creation d'une session persistante
    response = rs.get(url) #recolte d'info de l'url de la page
    soup = BeautifulSoup(response.text, "html.parser") #analyse de l'info dans une 'soupe'

    #on cherche l'information de chaque annonce 
    for annonce in soup.find_all("div", {"class": "apartment-card h-100"}):
        time.sleep(0.5)
        i = annonce.find("a",{"class":"card-bottom"})
        link = ("https://www.laforet.com" + str((i['href'])))  
        

        #recherche des informations par page en partant de 'link'
        page= requests.get(link)
        tree= html.fromstring(page.content)

        #On cherche la ville et le type de logement, contenus dans le titre
        try:
            title=tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[1]/div[1]/div[1]/div/h1/text()')
        except:
            title=[None]


        # Prix du bien
        try:
            price = tree.xpath("//div[@class='property__price w-100']/text()")
        except:
            price = [None]


        #surface
        try:
            surface = tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]/text()')
            if surface is None or len(surface) <=0:
                surface = [None]
        except:
            surface=[None]

        #nombre de pieces
        try:
            nb_pieces = tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[1]/div[2]/div[2]/div[2]/div[2]/text()')
            if nb_pieces is None or len(nb_pieces) <=0:
                nb_pieces = [None]
        except:
            nb_pieces=[None]

        #nombre de chambres
        try:
            chambres = tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[1]/div[2]/div[2]/div[2]/div[3]/text()')
            if chambres is None or len(chambres) <=0:
                chambres = [None]
        except:
            chambres =[None]

        #annee de construction
        try:
            annee_constr = tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[4]/div[4]/div/div/div[1]/div/span[2]/text()')
            if annee_constr is None or annee_constr == []:
                annee_constr=[None]
        except:
            annee_constr = [None]


        #les elements interieurs qui apportent ue valeur au bien
        #on commence par compter le nombre de features
        main_class = tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[4]/div[2]/div/div')
        if main_class is not None and len(main_class) > 0:
            sub_classes = main_class[0].xpath('.//div')
            nb_features_int=(len(sub_classes))

            #On recupere le texte correspondant aux features
            try:
                features_int=main_class[0].text_content()
            except:
                features_int=[None]
        else:
            nb_features_int = [0]
            features_int = ''

        #les elements exterieurs (typiquement piscine)
        main_class_ext = tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[4]/div[3]/div/div')
        if main_class_ext is not None and len(main_class_ext) > 0:
            sub_classes = main_class_ext[0].xpath('.//div')

            #On recupere le texte correspondant aux features
            try:
             features_ext=main_class_ext[0].text_content()
            except:
             features_ext=[None]
        else:
            features_ext = ''

        # Ajout d'une variable dummy portant sur la presence d'un terrain ou pas
        try:
            feature_ext2 = tree.xpath('//*[@id="main"]/div[3]/section[2]/div/div/div[1]/div[4]/div[1]/div/div/div[3]/div/span[2]')

            if len(feature_ext2)>0:
                text_content=feature_ext2[0].text_content()
                if "terrain" in text_content:
                    terrain=[1]
                else:
                    terrain=[0]
            else:
                terrain=[0]
        except:
            terrain=[None]

        

        print(f'link -> {link}')
        print(f'titr -> {title}')
        print(f'surface -> {surface}')
        print(f'nb_pieces -> {nb_pieces}')
        print(f'nb_chambres -> {chambres}')
        print(f'annee -> {annee_constr}')
        print(f'nb_features -> {nb_features_int}')
        print(f'features_int -> {features_int}')
        print(f'features_ext -> {features_ext}')
        print(f'terrain -> {terrain}')
        print(f'prix -> {price}')

        new_row = pd.DataFrame({'lien': [link], 'titre': title, 'surface': surface, 'nb_pieces': nb_pieces,
                            'nb_chambres': chambres,'annee_constr': annee_constr,'nb_features_int': nb_features_int,
                            'features_int': features_int,'features_ext': features_ext,'terrain': terrain, 'prix': price})
        print(new_row.head())
    
    # Append the new row to the DataFrame
        df = pd.concat([df, new_row], ignore_index=True)
    str_cols = ['lien', 'titre','surface','nb_pieces',
                            'nb_chambres','annee_constr', 'features_int', 'features_ext', 'prix']
    for x in str_cols:
        df[x] = df[x].str.strip()
    return df

In [17]:
df = scraper2(url="https://laforet.com/acheter/rechercher?next=60&filter%5Btypes%5D=house%2Capartment")

#df.to_csv('LaForet_listings.csv')


link -> https://www.laforet.com/agence-immobiliere/lagarennecolombes/acheter/la-garenne-colombes/appartement-2-pieces-21480289
titr -> ['Appartement T2 La Garenne-Colombes']
surface -> ['\n                    40.79 mÂ²\n                  ']
nb_pieces -> ['\n                    2 piÃ¨ce(s)\n                  ']
nb_chambres -> ['\n                    1 chambre(s)\n                  ']
annee -> ['\n                        Ann. const. : 1970\n                      ']
nb_features -> 6
features_int ->  
                        Cuisine IndÃ©pendante
                       
                        Avec ascenseur
                       
                        1 cave(s)
                      
features_ext -> 
terrain -> [0]
prix -> ['\n                290â\x80¯000Â\xa0â\x82¬']
                                                lien  \
0  https://www.laforet.com/agence-immobiliere/lag...   

                                titre  \
0  Appartement T2 La Garenne-Colombes   

                         

In [18]:
df.shape

(1100, 11)

In [16]:
#on telecharge les données on parquet pour avoir un stockage par colonnes, plus lisible sous pyspark
df.to_parquet('LaForet_listings.parquet')

In [1]:
from lxml import html
import requests

In [2]:
url_monde = "https://www.lemonde.fr/politique/article/2022/11/17/crise-energetique-les-limites-du-bouclier-tarifaire-sur-le-pouvoir-d-achat-des-francais_6150310_823448.html"

In [3]:
page= requests.get(url_monde)

In [4]:
tree= html.fromstring(page.content)

In [5]:
title=tree.xpath('//h1[@class="article__title"]/text()')

In [6]:
title

['Crise énergétique\xa0: les limites du bouclier tarifaire sur le pouvoir d’achat des Français']

In [7]:
article_extrait=tree.xpath('//p[@class="article__paragraph "]/text()') 

In [8]:
article_extrait

['A situation inédite, réponse inédite. Confronté à l’emballement de l’inflation après la reprise économique post-Covid-19, puis dans le sillage de la guerre en Ukraine, l’exécutif a mis en place des mesures massives pour empêcher les factures des Français de flamber\xa0: le bouclier tarifaire, qui gèle les prix du gaz et plafonne à 4\xa0% ceux de l’électricité – avant une hausse contenue à 15\xa0% en\xa02023 – et des remises à la pompe jusqu’à la fin de l’année.',
 'Cette réponse était-elle la plus adaptée au regard des inégalités de revenus, mais aussi des différents modes de vie et de consommation des Français\xa0? Aurait-il fallu dessiner différemment les dispositifs de lutte contre le choc énergétique pour préserver le pouvoir d’achat des ménages et la croissance\xa0?',
 'C’est pour répondre à ces questions que l’Institut des politiques publiques (IPP), un organisme de recherche indépendant, a choisi d’orienter sur le choc énergétique son étude budgétaire annuelle, qui devait être

In [9]:
article=tree.xpath('//article[@class="article__content  old__article-content-single"]/text()')

In [10]:
article

['                  ',
 '           ',
 '                           ',
 '               ',
 '          ',
 '           ',
 '          ',
 '          ',
 '          ',
 '        ',
 '    ']